In [ ]:
# created on Dec 24, 2020
# modified on April 12, 2022
# @author:          Bo Zhao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

In [1]:
import tweepy, json, time
import pandas as pd
from google.colab import files
# Create data on to Google Drive
from google.colab import drive
# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.result = []
        self.f = file
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            # print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    row = {
                        'id': id,
                        'username': username,
                        'created_at': created_at,
                        'lng': lng,
                        'lat': lat,
                        'text': text
                    }
                    print(row)
                    self.result.append(row)
                else:
                    pass
        else:
            df = pd.DataFrame(self.result)
            df.to_csv(self.f, index=False)
            # download the csv to your local computer
            files.download(self.f)
            print("the csv has been downloaded to your local computer. The program has been completed successfully.")
            return False

In [3]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = '/gdrive/My Drive/GEOG 458/geotweets.csv'

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "eyU93kxcG9KX51ONxA5hXDd2S"
    consumer_secret = "b0mWDYL8o143gthsym2xBrsB6XW9B0DD7p9DmrwvnSSX1T6ZfB"
    access_token = "1517587443389673472-oJVUALo8O5HyoGds7ldM7rZochhAB0"
    access_token_secret = "fXsbt59wfOMh7vscYRQiK5CA5cP64wEa6ZyCqmU0Q5uSQ"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=600, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS, languages=['en'], encoding="utf-8")

{'id': 1517592467150888961, 'username': 'programmehq', 'created_at': 'Fri Apr 22 19:53:34 +0000 2022', 'lng': -117.88737722, 'lat': 33.87445803, 'text': '“Yes to answer your question we will have the Taylor Swift-The Lakes 7” tomorrow” @ Programme Skate &amp; Sound https://t.co/4DmlehxwGi'}
{'id': 1517592467599679490, 'username': 'rapSkip_Bayless', 'created_at': 'Fri Apr 22 19:53:34 +0000 2022', 'lng': -73.9685415, 'lat': 40.780709, 'text': '@NYCTSubway my j train was on the bridge for like 15 minutes. must have been some traffic.'}
{'id': 1517592467981447168, 'username': 'JordanHallWX', 'created_at': 'Fri Apr 22 19:53:34 +0000 2022', 'lng': -99.6809025, 'lat': 41.500729500000006, 'text': '@BrandonCopicWx @BradArnoldWX Goodluck'}
{'id': 1517592468555919361, 'username': 'notjbrito19', 'created_at': 'Fri Apr 22 19:53:34 +0000 2022', 'lng': -116.5172895, 'lat': 33.8250795, 'text': 'I absolutely hate the new Twitter update'}
{'id': 1517592470732820483, 'username': 'wtfsab', 'created_at': '

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

the csv has been downloaded to your local computer. The program has been completed successfully.
